In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential

network = Sequential([
    layers.Dense(3, activation=None),
    layers.ReLU(),
    layers.Dense(2, activation=None),
    layers.ReLU()
    ])
x = tf.random.normal([2,4])  # batch_size, input_dim)`.

print(network(x))  # 输入从第一层开始，逐层传播至最末层
print(network.summary())


tf.Tensor(
[[0.        1.3153355]
 [0.        0.       ]], shape=(2, 2), dtype=float32)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  15        
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  8         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________
None


- 注意参数量的计算

15= 4*3+3

8 = 3*2+2

In [2]:
# 也可以通过 add()方法
layers_num=2  # 堆叠两次
network = Sequential([])
for _ in range(layers_num):
    network.add(layers.Dense(3))
    network.add(layers.ReLU())
network.build(input_shape=(None, 4))
print(network.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              multiple                  15        
_________________________________________________________________
re_lu_2 (ReLU)               multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  12        
_________________________________________________________________
re_lu_3 (ReLU)               multiple                  0         
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
# 打印网络的待优化参数名
for p in network.trainable_variables:
    print(p.name, p.shape)

dense_2/kernel:0 (4, 3)
dense_2/bias:0 (3,)
dense_3/kernel:0 (3, 3)
dense_3/bias:0 (3,)


## 模型装配、训练与测试

In [16]:


# 创建5层的全连接层网络
network = Sequential([layers.Dense(256, activation='relu'),
                      layers.Dense(128, activation='relu'), 
                      layers.Dense(64, activation='relu'), 
                      layers.Dense(32, activation='relu'), 
                      layers.Dense(10)])
network.build(input_shape=(None, 28*28)) 
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             multiple                  200960    
_________________________________________________________________
dense_15 (Dense)             multiple                  32896     
_________________________________________________________________
dense_16 (Dense)             multiple                  8256      
_________________________________________________________________
dense_17 (Dense)             multiple                  2080      
_________________________________________________________________
dense_18 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [7]:
28*28*256+256
257*128

32896

In [17]:
# 损失函数
from tensorflow.keras import optimizers, losses
# 采用Adam优化器，学习率为0.01;采用交叉熵损失函数，包含Softmax 
network.compile(optimizer=optimizers.Adam(lr=0.01),
                loss=losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'] # 设置测量指标为准确率
               )

In [ ]:
# 模型训练
# 指定训练集为train_db，验证集为val_db,训练5个epochs，每2个epoch验证一次 
# 返回训练信息保存在history中
from tensorflow.keras import datasets
(x, y), (x_val, y_val) = datasets.mnist.load_data()
train_db = tf.data.Dataset.from_tensor_slices((x,y))
val_db = tf.data.Dataset.from_tensor_slices((x_val,y_val))

validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))

history = network.fit(train_db, epochs=5, validation_data=val_db, validation_freq=2)

history.history

In [ ]:
# 模型测试
x,y= next(iter(db_test))
print('predict x:', x.shape)
out=network.predict(x)  # 模型预测



## 模型保存与加载

### 1.张量方式
model.save_weights(path)

In [ ]:
# 保存模型参数到文件上
network.save_weights('weights.ckpt') 
print('saved weights.')
del network # 删除网络对象

# 重新创建相同的网络结构
network = Sequential([layers.Dense(256, activation='relu'),
                      layers.Dense(128, activation='relu'),
                      layers.Dense(64, activation='relu'), 
                      layers.Dense(32, activation='relu'), 
                      layers.Dense(10)])
network.compile(optimizer=optimizers.Adam(lr=0.01),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy']
               )
# 从参数文件中读取数据并写入当前网络
network.load_weights('weights.ckpt')
print('loaded weights!')

### 2. 网络方式

In [19]:
# 直接加载
# 保存模型结构与模型参数到文件 
network.save('model.h5') 
print('saved total model.')
del network # 删除网络对象

# 从文件恢复网络结构与网络参数
network = tf.keras.models.load_model('model.h5')

saved total model.


### 3. SaveModel 方式

In [ ]:
# 当需要将模型部署到其他平台时，采用此方式更具有平台无关性
# 保存模型结构与模型参数到文件
tf.keras.experimental.export_saved_model(network, 'model-savedmodel')
print('export saved model.')
del network # 删除网络对象


# 从文件恢复网络结构与网络参数
network = tf.keras.experimental.load_from_saved_model('model-savedmodel')